In [1]:
# -*-coding: utf-8 -*-
import feedparser
import string
import sqlite3 as sqlite
import sys
import newspaper
from newspaper import Article, news_pool
import threading, time
from time import mktime
from datetime import datetime
import tweepy

In [2]:
rssSources = {
    'Al Jazeera':'https://www.aljazeera.com/xml/rss/all.xml',
    'BBC News':'http://feeds.bbci.co.uk/news/world/rss.xml',
    'The New York Times':'https://www.nytimes.com/svc/collections/v1/publish/https://www.nytimes.com/section/world/rss.xml',
    'Global Issues':'http://www.globalissues.org/news/feed',
    'Reuters':'http://feeds.reuters.com/Reuters/worldNews',
    'Daily Express':'http://feeds.feedburner.com/daily-express-world-news',
    'The Guardian':'https://www.theguardian.com/world/rss',
    'The Independent': 'http://www.independent.co.uk/news/world/rss',
    'The Sun Daily': 'http://www.thesundaily.my/rss/world',
    'CNBC': 'https://www.cnbc.com/id/100727362/device/rss/rss.html'
}
twitCred = {
    'consumer_key':'1OgJMHusv7DjyvkqCJADEt1Xn',
    'consumer_secret':'rfj2mHi8qKS1gQtrl2QBoGmXK6jcTi0ann5R3Kn877HCQBNf5n',
    'access_token':'1134350418-s6h7wX2M9GGsM4Me67RIcmt4ZjBeYrl3R1heiFe',
    'access_token_secret':'elN0VJ8vo8Oioh5pTZeGcu5MHFCNzudXB4ktTM5cSh4kD'
}
twitSources = {
    'CBS News':'@CBSNews',
    'Fox News': '@FoxNews',
    'BuzzFeed News': '@BuzzFeedNews',
    'The Associated Press':'@AP'
}
webSources = {
    'CNN':'https://edition.cnn.com/specials/last-50-stories',
    'Global News': 'https://globalnews.ca/world/',
    'AU News': 'https://www.news.com.au/world/breaking-news'
}
instaSources = {
    'TODAY': 'https://www.instagram.com/todayonline/',
    'The Economist': 'https://www.instagram.com/theeconomist/'
}

In [3]:
class setInterval :
    def __init__(self,interval,action) :
        self.interval=interval
        self.action=action
        self.stopEvent=threading.Event()
        thread=threading.Thread(target=self.__setInterval)
        thread.start()

    def __setInterval(self) :
        nextTime=time.time()+self.interval
        while not self.stopEvent.wait(nextTime-time.time()) :
            nextTime+=self.interval
            self.action()

    def cancel(self) :
        self.stopEvent.set()

In [4]:
def getRSSObj(x, value):
    tags = time = author = ''
    if hasattr(x, 'tags'):
        for tag in x.tags:
            if(tags == ''):
                tags = tag.term
            else:
                tags += '|' + tag.term
                
    if hasattr(x, 'published_parsed'):
        time = x.published_parsed
    else:
        time = x.updated_parsed
        
    if hasattr(x, 'author'):
        author = x.author
        
    time = datetime.fromtimestamp(mktime(time))

    return (x.title,x.link,x.summary,time,author,tags,value)

def getTweetObj(x, key, value):
    urls = tags = ''
    for url in x.entities['urls']:
        if(urls == ''):
            urls = url['url']
        else:
            urls += '|' + url['url']
        
    for tag in x.entities['hashtags']:
        if(tags == ''):
            tags = tag['text']
        else:
            tags += '|' + tag['text']
        
    return (x.id,x.full_text,urls,x.created_at,tags,value,key)

In [5]:
def cleanDB():
    with sqlite.connect(r'news.db') as con: 
        cur = con.cursor()
        
        cur.execute("drop table if exists rss;")
        cur.execute("create table rss(id integer primary key autoincrement, title text unique, link text unique, description text, published_at datetime, authors text, tags text, source text)")
    
        cur.execute("drop table if exists web;")
        cur.execute("create table web(id integer primary key autoincrement, title text unique, link text unique, description text, published_at datetime, authors text, tags text, source text)")
        
        cur.execute("drop table if exists twitter;")
        cur.execute("create table twitter(id integer primary key, tweet text, link text, published_at datetime, hashtags text, sourceHandle text, source text)")
        
        cur.execute("drop table if exists instagram;")
        cur.execute("create table instagram(id integer primary key, caption text, link text, published_at datetime, sourceHandle text, source text)")
        
        con.commit()

#cleanDB()

In [6]:
# Parses the RSS feed from RSS
def parseRSS():
    rssNews = []
    for key, value in rssSources.items():
        d = feedparser.parse(value)
        rssNews += [getRSSObj(x, key) for x in d['items']]

    with sqlite.connect(r'news.db') as con: 
        cur = con.cursor()
        cur.executemany("insert or ignore into rss(title,link,description,published_at,authors,tags,source) values(?,?,?,?,?,?,?)", rssNews)
        con.commit()
        
#parseRSS()

In [7]:
# run after every 5 min interval
interRss=setInterval(1800,parseRSS)

# will stop interval in 1hr
trss=threading.Timer(21600,interRss.cancel)
trss.start()

In [8]:
since_id = {}
for key, value in twitSources.items():
    since_id[key] = None
    
def getTweets():
    twitNews = []
    auth = tweepy.OAuthHandler(twitCred['consumer_key'], twitCred['consumer_secret'])
    auth.set_access_token(twitCred['access_token'], twitCred['access_token_secret'])
    api = tweepy.API(auth,wait_on_rate_limit=True)
    
    for key, value in twitSources.items():
        allTweets = tweepy.Cursor(
            api.user_timeline,
            id=value,
            screen_name=value,
            trim_user=True,
            include_rts=False,
            tweet_mode='extended',
            since_id = since_id[key]
            ).items(100)
        
        indTwit = [getTweetObj(x, key, value) for x in allTweets]
        if(len(indTwit)):
            since_id[key] = indTwit[0][0]
            twitNews += indTwit

    with sqlite.connect(r'news.db') as con: 
        cur = con.cursor()
        cur.executemany("insert or ignore into twitter values(?,?,?,?,?,?,?)", twitNews)
        con.commit()
        
#getTweets()

In [9]:
# run after every 5 min interval
interTwit=setInterval(1800,getTweets)

# will stop interval in 1hr
ttwit=threading.Timer(21600,interTwit.cancel)
ttwit.start()

In [10]:
# Parses the WebPages from News Websites
def scrapWebsites():
    webNews = []
    for key, value in webSources.items():
        paper = newspaper.build(value, memoize_articles=True, fetch_images=False)

        count = 1
        for content in paper.articles:
            try:
                content.download()
                content.parse()
            except Exception as e:
                #print(e)
                continue

            if content.publish_date is None:
                continue
                
            authors = tags = ''
            for author in content.authors:
                if(authors == ''):
                    authors = author
                else:
                    authors += '|' + author
                    
            for tag in content.tags:
                if(tags == ''):
                    tags = tag
                else:
                    tags += '|' + tag
            webNews.append((content.title,content.url,content.text,content.publish_date,authors,tags,key))

            #print(count, "articles downloaded from", key, "using newspaper, url: ", content.url)
            count = count + 1

            if(count>20):
                break
        
    with sqlite.connect(r'news.db') as con: 
        cur = con.cursor()
        cur.executemany("insert or ignore into web(title,link,description,published_at,authors,tags,source) values(?,?,?,?,?,?,?)", webNews)
        con.commit()
        
#scrapWebsites()

In [11]:
# run after every 5 min interval
interWeb=setInterval(1800,scrapWebsites)

# will stop interval in 1hr
tweb=threading.Timer(21600,interWeb.cancel)
tweb.start()

In [12]:
import instagram
from instagram import InstagramScraper

def getInstaPosts():
    instaPosts = []
    IS = InstagramScraper()
    for key, value in instaSources.items():
        results = IS.profile_page_recent_posts(value)
        for result in results:
            instaId = int(result['id'])
            instaURL = result['display_url']
            instaDate = datetime.fromtimestamp(result['taken_at_timestamp'])
            instaHandle = result['owner']['username']
            instaCaption = result['edge_media_to_caption']['edges'][0]['node']['text']
            instaPosts.append((instaId, instaCaption, instaURL, instaDate, instaHandle, key))
      
    with sqlite.connect(r'news.db') as con: 
        cur = con.cursor()
        cur.executemany("insert or ignore into instagram values(?,?,?,?,?,?)", instaPosts)
        con.commit()
        
#getInstaPosts()

In [13]:
# run after every 5 min interval
interInsta=setInterval(1800,getInstaPosts)

# will stop interval in 1hr
tinsta=threading.Timer(21600,interInsta.cancel)
tinsta.start()